<a href="https://colab.research.google.com/github/Sugam1530/Specialized-LLM-Bot-Using-Pre-Trained-Models/blob/main/IndustryGPT_Specialized_LLM_Bot_Using_Pre_Trained_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project name - DeepFER: Facial Emotion Recognition Using Deep Learning Deep Learning for Computer Vision

# **Project Summary -**

This project focuses on building a domain-specific AI chatbot for e-commerce customer support by fine-tuning the T5 transformer model and deploying it via a lightweight web interface using Streamlit and ngrok. The aim is to provide quick, automated, and accurate responses to frequently asked customer queries such as order tracking, delivery options, return/exchange policies, and payment methods.

The chatbot is based on the t5-small model from HuggingFace Transformers, which is known for its text-to-text generation capability. It is trained on a custom dataset tailored specifically for customer service scenarios. The dataset is structured with input prompts in the format "User: <question>\nBot:" and the corresponding expected response. This ensures the model learns not just the content but the conversational structure required for role-based interactions.

To prepare the dataset, each example was tokenized using the T5 tokenizer with padding and truncation applied to standardize input lengths. The preprocessing logic was carefully designed to avoid repetition of prefixes such as "prompt:" to maintain clarity in the training data. The fine-tuning was conducted using the HuggingFace Trainer API within Google Colab. Key hyperparameters such as the number of training epochs (set to 50) and learning rate (1e-4) were adjusted to improve model convergence and response quality.

During evaluation, the model initially faced challenges such as blank or irrelevant responses. These issues were resolved by refining the prompt structure, increasing the number of training epochs, and removing unsupported generation parameters like temperature and top_p for greedy decoding. After fine-tuning, the model showed improved performance and was able to generate relevant and fluent answers to various test inputs from the e-commerce domain.

For deployment, a Streamlit-based web interface was developed that allows users to interact with the chatbot in real-time. The app takes user input, processes it through the fine-tuned model, and displays the response immediately. Since Google Colab does not support web hosting, pyngrok was integrated to create a secure public tunnel that exposes the Streamlit app to the internet. This setup enables full interaction with the chatbot from any device without requiring backend hosting infrastructure.

The final application provides a functional prototype of an AI assistant capable of handling routine customer queries. It has potential real-world applications in small to medium-sized e-commerce platforms where human customer service may be limited. Future enhancements could include multilingual support, intent classification, user history tracking, and integration with real databases for dynamic responses.

In summary, this project demonstrates how transformer-based NLP models can be effectively fine-tuned for narrow domains and deployed using accessible tools like Streamlit and Google Colab. It offers a practical, scalable, and cost-efficient solution to enhance customer service through automation.



# **GitHub Link -**

In [1]:
!pip install transformers datasets sentencepiece
print("✅ Done! Now manually click: Runtime > Restart runtime")

✅ Done! Now manually click: Runtime > Restart runtime


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import numpy as np
import torch

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [3]:
# Read the uploaded CSV
df = pd.read_csv("retail_bot_dataset.csv")
df.head()

,prompt,response
0,User: How do I track my order?\nBot:,You can track your order in the 'My Orders' se...
1,User: What is your return policy?\nBot:,You can return products within 7 days of deliv...
2,User: How can I cancel an order?\nBot:,"Go to 'My Orders', select the item, and click ..."
3,User: Do you offer free shipping?\nBot:,"Yes, we offer free shipping on orders above ₹499."
4,User: How do I make a payment?\nBot:,"You can pay using debit/credit cards, UPI, net..."


In [4]:
from datasets import Dataset, DatasetDict

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 80% train, 20% test split
split_idx = int(0.8 * len(df))
train_df = df.iloc[:split_idx]
test_df = df.iloc[split_idx:]

# Convert to HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 16
    })
    test: Dataset({
        features: ['prompt', 'response'],
        num_rows: 4
    })
})

In [6]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_checkpoint = "t5-small"
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [25]:
def preprocess(batch):
    # ❌ Remove "prompt: " if it's already in your prompt column
    inputs = batch["prompt"]  # No need to add "prompt: "
    targets = batch["response"]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=64, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
dataset["train"][0]

{'prompt': 'User: How do I track my order?\nBot:',
 'response': "You can track your order in the 'My Orders' section after logging in."}

In [26]:
tokenized_dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [29]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [84]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-4,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    eval_steps=500,
    do_eval=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-84-658202091>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [85]:
trainer.train()

Step,Training Loss
10,0.031100
20,0.033700


TrainOutput(global_step=20, training_loss=0.03238178491592407, metrics={'train_runtime': 138.5893, 'train_samples_per_second': 1.154, 'train_steps_per_second': 0.144, 'total_flos': 5413672058880.0, 'train_loss': 0.03238178491592407, 'epoch': 10.0})

In [86]:
model.save_pretrained("./finetuned-t5-bot")
tokenizer.save_pretrained("./finetuned-t5-bot")

('./finetuned-t5-bot/tokenizer_config.json',
 './finetuned-t5-bot/special_tokens_map.json',
 './finetuned-t5-bot/spiece.model',
 './finetuned-t5-bot/added_tokens.json')

In [92]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load from fine-tuned directory
model = T5ForConditionalGeneration.from_pretrained("./finetuned-t5-bot")
tokenizer = T5Tokenizer.from_pretrained("./finetuned-t5-bot")

def chat_with_bot(user_input):
    prompt = f"User: {user_input}\nBot:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_length=64,
        do_sample=True,
        temperature=0.9,
        top_p=0.9,
        top_k=50,
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [93]:
print(chat_with_bot("How do I track my order?"))

You can track your order in the 'My Orders' section after logging in.


In [90]:
def chat_loop():
    print("🟢 ChatBot is ready! Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Bot: 👋 Goodbye!")
            break

        # Pass fully formatted prompt
        full_prompt = f"User: {user_input}\nBot:"
        response = chat_with_bot(full_prompt)
        print(f"Bot: {response}\n")


In [91]:
chat_loop()

🟢 ChatBot is ready! Type 'exit' to quit.

You: Can I change my delivery address?


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Bot: Yes, please visit our website at checkout in the 'My Delivery Address' section.



KeyboardInterrupt: Interrupted by user

Web App making using Streamlit


In [94]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


In [95]:
import streamlit as st
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load the fine-tuned model and tokenizer
@st.cache_resource
def load_model():
    model = T5ForConditionalGeneration.from_pretrained("./finetuned-t5-bot")
    tokenizer = T5Tokenizer.from_pretrained("./finetuned-t5-bot")
    model.eval()
    return model, tokenizer

model, tokenizer = load_model()

# Chat function
def chat_with_bot(user_input):
    prompt = f"User: {user_input}\nBot:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=64,
            num_beams=4,
            early_stopping=True
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip()

# Streamlit UI
st.set_page_config(page_title="T5 ChatBot", page_icon="🤖", layout="centered")
st.title("🤖 Fine-Tuned T5 ChatBot")
st.markdown("Ask me anything about your service!")

# Session state for chat history
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# Input field
user_input = st.text_input("You:", key="user_input")

if user_input:
    bot_response = chat_with_bot(user_input)
    st.session_state.chat_history.append(("You", user_input))
    st.session_state.chat_history.append(("Bot", bot_response))
    st.text_input("You:", value="", key="user_input", placeholder="Ask another question...")

# Display chat history
for sender, msg in st.session_state.chat_history:
    if sender == "You":
        st.markdown(f"🧑‍💬 **{sender}**: {msg}")
    else:
        st.markdown(f"🤖 **{sender}**: {msg}")

2025-06-12 15:44:22.493 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 15:44:23.184 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-12 15:44:23.192 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 15:44:23.193 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 15:44:23.927 Thread 'Thread-19': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 15:44:23.945 Thread 'Thread-19': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 15:44:24.421 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 15:44:24.425 Thread 'MainThread': miss

In [97]:
!pip install pyngrok

In [100]:
from pyngrok import ngrok

# Replace with your actual authtoken
NGROK_AUTH_TOKEN = "2V1dW3QU9dMtAmG2PST5tArWbtq_54ssb8xUdLtCh5Z5uvFhL"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [102]:
%%writefile app.py
import streamlit as st
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./finetuned-t5-bot")
tokenizer = T5Tokenizer.from_pretrained("./finetuned-t5-bot")

def chat_with_bot(user_input):
    input_text = "prompt: " + user_input
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs, max_length=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Streamlit UI
st.title("🛍️ E-Commerce Support Bot")
user_input = st.text_input("Ask your question:")

if user_input:
    response = chat_with_bot(user_input)
    st.write("🤖 Bot:", response)


Overwriting app.py


In [103]:
import time

# Kill any existing tunnel
ngrok.kill()

# Start streamlit in background
!streamlit run app.py &>/content/logs.txt &

# Wait for Streamlit to launch
time.sleep(3)

# Open ngrok tunnel on http://localhost:8501
public_url = ngrok.connect("http://localhost:8501")
print(f"🔗 Click here to open your chatbot: {public_url}")

🔗 Click here to open your chatbot: NgrokTunnel: "https://c5e4-34-16-234-137.ngrok-free.app" -> "http://localhost:8501"


# Conclusion -

This project successfully demonstrates how transformer models like T5 can be fine-tuned and deployed for practical use in customer service automation. By focusing on a specific domain—e-commerce—the chatbot achieves high relevance and fluency in responses. The use of accessible tools like Google Colab, HuggingFace Transformers, Streamlit, and ngrok ensures that the entire pipeline is cost-effective and replicable for small businesses or student projects.

In essence, this chatbot is not just a technical exercise but a practical proof-of-concept for scalable, intelligent customer interaction. It bridges the gap between cutting-edge NLP research and real-world application, offering businesses a way to enhance user experience while optimizing operational efficiency.

